## Setup

In [ ]:
%pip install mlflow>=2.11.0 -q -U
%pip install transformers>=4.34.0 -q -U


In [ ]:
%env TOKENIZERS_PARLLELISM=false

import warnings 
warnings.filterwarnings("ignore", category=UseWarning)

## Conversational AI with MLflow and DialoGPT

In [ ]:
import transformer 
import mlflow 

conversational_pipeline = transformer.pipeline(model="microsoft/DialoGPT-medium")

signature = mlflow.models.infer_signature(
    "Hi there, chatbot!",
    mlflow.transformers.generate_signature_output(conversational_pipeline, "Hi there, chatbot!"),
    
)

In [ ]:
mlflow.set_experiment("Conversational")


In [ ]:
with mlflow.start_run():
    model_info = mlflow.transformers.log_model(
        transformers_model=conversational_pipeline,
        artifact_path="chatbot",
        task="conversational",
        signature=signature,
        input_example="A clever and witty question",
    )

### 1. Loading and interacting with the Chatbot model

In [ ]:
chatbot = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)

first = chatbot.predict("What is the best way to get to Atarctica")

## Deploying a Transformer model as an OpenAI-compatible Chatbot

In [ ]:
from transformers import pipeline 

import mlflow 

generator = pipeline(
    "text-generation",
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
)

mlflow.transformers.save_model(
    path="tinyllama-text-generation",
    transformers_model=generator,
    task="text-generation"
)

In [ ]:
model= mlflow.pyfunc.load_model("tinyllama-text-generation")

model.metadata.signature


In [ ]:
messages = [{"role":"user", "content":"Write me a hello world program by python."}]

prompt = generator.tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

model.predict(prompt)

In [ ]:
mlflow.transformers.save_model(
    path="tinyllama-chat",
    transformer_model=generator,
    task="llm/v1/chat"
)

In [ ]:
model = mlflow.pyfunc.load_model("tinyllama-chat")

model.metadata.signature

## Serving the chat Model

In [ ]:
%%bash 
mlflow models serve -m tinyllama-chat 

In [ ]:
%%sh 
curl http://localhost:5000/invocations -H 'Content-Type: application/json' -d '{"messages":[{"role": "user", "content": "Write me a hello world program in python"}]}' | jq